In [ ]:
# CARLA INTEGRATION GUIDE
# How to use SDXL generated rural driving data with CARLA simulator

import carla
import numpy as np
import cv2
from PIL import Image
import random
import time
import os
import json

print("🚗 CARLA INTEGRATION GUIDE FOR SDXL DATA")
print("=" * 50)

# ===== PART 1: LOADING SDXL DATASET =====
class SDXLDatasetLoader:
    """Load and manage SDXL generated rural driving dataset"""
    
    def __init__(self, dataset_path):
        self.dataset_path = dataset_path
        self.images = []
        self.metadata = {}
        self.load_dataset()
    
    def load_dataset(self):
        """Load SDXL dataset from saved directory"""
        print(f"📂 Loading SDXL dataset from: {self.dataset_path}")
        
        # Load metadata
        metadata_path = os.path.join(self.dataset_path, 'metadata.json')
        if os.path.exists(metadata_path):
            with open(metadata_path, 'r') as f:
                self.metadata = json.load(f)
            print(f"✅ Loaded metadata: {self.metadata['dataset_info']['num_images']} images")
        
        # Load numpy dataset
        numpy_path = os.path.join(self.dataset_path, 'synthetic_datasets.npy')
        if os.path.exists(numpy_path):
            self.numpy_data = np.load(numpy_path)
            print(f"✅ Loaded numpy data: {self.numpy_data.shape}")
        
        # Load individual images
        images_dir = os.path.join(self.dataset_path, 'images')
        if os.path.exists(images_dir):
            image_files = sorted([f for f in os.listdir(images_dir) if f.endswith('.png')])
            
            for img_file in image_files:
                img_path = os.path.join(images_dir, img_file)
                img = Image.open(img_path)
                self.images.append(np.array(img))
            
            print(f"✅ Loaded {len(self.images)} individual images")
    
    def get_random_image(self):
        """Get random SDXL generated image"""
        if self.images:
            return random.choice(self.images)
        return None
    
    def get_image_by_index(self, index):
        """Get specific SDXL image by index"""
        if 0 <= index < len(self.images):
            return self.images[index]
        return None

# ===== PART 2: CARLA SETUP AND CONNECTION =====
class CARLAIntegration:
    """Integrate SDXL data with CARLA simulator"""
    
    def __init__(self, host='localhost', port=2000):
        self.host = host
        self.port = port
        self.client = None
        self.world = None
        self.vehicle = None
        self.camera = None
        self.sdxl_loader = None
        
    def connect_to_carla(self):
        """Connect to CARLA simulator"""
        try:
            print(f"🔌 Connecting to CARLA at {self.host}:{self.port}")
            self.client = carla.Client(self.host, self.port)
            self.client.set_timeout(10.0)
            
            # Get world
            self.world = self.client.get_world()
            print(f"✅ Connected to CARLA world: {self.world.get_map().name}")
            
            return True
            
        except Exception as e:
            print(f"❌ Failed to connect to CARLA: {e}")
            print("💡 Make sure CARLA simulator is running")
            return False
    
    def load_sdxl_dataset(self, dataset_path):
        """Load SDXL dataset for comparison"""
        self.sdxl_loader = SDXLDatasetLoader(dataset_path)
        return self.sdxl_loader is not None
    
    def spawn_vehicle(self, vehicle_type='vehicle.tesla.model3'):
        """Spawn a vehicle in CARLA"""
        try:
            # Get spawn points
            spawn_points = self.world.get_map().get_spawn_points()
            
            if not spawn_points:
                print("❌ No spawn points available")
                return False
            
            # Choose random spawn point
            spawn_point = random.choice(spawn_points)
            
            # Get vehicle blueprint
            blueprint_library = self.world.get_blueprint_library()
            vehicle_bp = blueprint_library.find(vehicle_type)
            
            # Spawn vehicle
            self.vehicle = self.world.spawn_actor(vehicle_bp, spawn_point)
            print(f"✅ Spawned vehicle: {vehicle_type}")
            
            return True
            
        except Exception as e:
            print(f"❌ Failed to spawn vehicle: {e}")
            return False
    
    def setup_camera(self, width=1024, height=1024):
        """Setup camera sensor on vehicle"""
        try:
            # Get camera blueprint
            blueprint_library = self.world.get_blueprint_library()
            camera_bp = blueprint_library.find('sensor.camera.rgb')
            
            # Set camera attributes to match SDXL resolution
            camera_bp.set_attribute('image_size_x', str(width))
            camera_bp.set_attribute('image_size_y', str(height))
            camera_bp.set_attribute('fov', '90')
            
            # Camera transform (attach to vehicle)
            camera_transform = carla.Transform(
                carla.Location(x=2.0, z=1.4),  # Front of vehicle, driver height
                carla.Rotation(pitch=0.0)       # Level view
            )
            
            # Spawn camera
            self.camera = self.world.spawn_actor(
                camera_bp, 
                camera_transform, 
                attach_to=self.vehicle
            )
            
            print(f"✅ Camera setup: {width}x{height}")
            return True
            
        except Exception as e:
            print(f"❌ Failed to setup camera: {e}")
            return False

# ===== PART 3: DATA COMPARISON AND ANALYSIS =====
class DataComparison:
    """Compare CARLA captured data with SDXL generated data"""
    
    def __init__(self, carla_integration, sdxl_loader):
        self.carla = carla_integration
        self.sdxl = sdxl_loader
        self.carla_images = []
        self.comparison_results = []
    
    def capture_carla_image(self):
        """Capture single image from CARLA"""
        if not self.carla.camera:
            print("❌ No camera available")
            return None
        
        # Set up image capture
        image_data = None
        
        def process_image(image):
            nonlocal image_data
            # Convert CARLA image to numpy array
            array = np.frombuffer(image.raw_data, dtype=np.dtype("uint8"))
            array = np.reshape(array, (image.height, image.width, 4))  # RGBA
            array = array[:, :, :3]  # Remove alpha channel
            image_data = array
        
        # Listen for image
        self.carla.camera.listen(process_image)
        
        # Wait for image capture
        time.sleep(0.1)
        
        # Stop listening
        self.carla.camera.stop()
        
        if image_data is not None:
            self.carla_images.append(image_data)
            print(f"📸 Captured CARLA image: {image_data.shape}")
        
        return image_data
    
    def compare_images(self, carla_img, sdxl_img):
        """Compare CARLA and SDXL images"""
        
        # Resize to same dimensions if needed
        if carla_img.shape != sdxl_img.shape:
            sdxl_img = cv2.resize(sdxl_img, (carla_img.shape[1], carla_img.shape[0]))
        
        # Calculate similarity metrics
        # 1. Structural Similarity (SSIM)
        carla_gray = cv2.cvtColor(carla_img, cv2.COLOR_RGB2GRAY)
        sdxl_gray = cv2.cvtColor(sdxl_img, cv2.COLOR_RGB2GRAY)
        
        # Simple SSIM approximation
        def calculate_ssim(img1, img2):
            mu1 = cv2.GaussianBlur(img1.astype(np.float64), (11, 11), 1.5)
            mu2 = cv2.GaussianBlur(img2.astype(np.float64), (11, 11), 1.5)
            
            mu1_sq = mu1 * mu1
            mu2_sq = mu2 * mu2
            mu1_mu2 = mu1 * mu2
            
            sigma1_sq = cv2.GaussianBlur(img1.astype(np.float64) * img1.astype(np.float64), (11, 11), 1.5) - mu1_sq
            sigma2_sq = cv2.GaussianBlur(img2.astype(np.float64) * img2.astype(np.float64), (11, 11), 1.5) - mu2_sq
            sigma12 = cv2.GaussianBlur(img1.astype(np.float64) * img2.astype(np.float64), (11, 11), 1.5) - mu1_mu2
            
            C1 = (0.01 * 255) ** 2
            C2 = (0.03 * 255) ** 2
            
            numerator = (2 * mu1_mu2 + C1) * (2 * sigma12 + C2)
            denominator = (mu1_sq + mu2_sq + C1) * (sigma1_sq + sigma2_sq + C2)
            
            return np.mean(numerator / denominator)
        
        ssim_score = calculate_ssim(carla_gray, sdxl_gray)
        
        # 2. Color similarity
        carla_mean_color = np.mean(carla_img, axis=(0, 1))
        sdxl_mean_color = np.mean(sdxl_img, axis=(0, 1))
        color_similarity = 1.0 - np.mean(np.abs(carla_mean_color - sdxl_mean_color)) / 255.0
        
        # 3. Brightness similarity
        carla_brightness = np.mean(carla_gray) / 255.0
        sdxl_brightness = np.mean(sdxl_gray) / 255.0
        brightness_similarity = 1.0 - abs(carla_brightness - sdxl_brightness)
        
        comparison = {
            'ssim': ssim_score,
            'color_similarity': color_similarity,
            'brightness_similarity': brightness_similarity,
            'overall_similarity': (ssim_score + color_similarity + brightness_similarity) / 3
        }
        
        return comparison
    
    def run_comparison_session(self, num_comparisons=10):
        """Run comparison session between CARLA and SDXL data"""
        print(f"🔄 Starting comparison session: {num_comparisons} comparisons")
        
        results = []
        
        for i in range(num_comparisons):
            print(f"\n📊 Comparison {i+1}/{num_comparisons}")
            
            # Capture CARLA image
            carla_img = self.capture_carla_image()
            if carla_img is None:
                continue
            
            # Get random SDXL image
            sdxl_img = self.sdxl.get_random_image()
            if sdxl_img is None:
                continue
            
            # Compare images
            comparison = self.compare_images(carla_img, sdxl_img)
            results.append(comparison)
            
            print(f"   SSIM: {comparison['ssim']:.3f}")
            print(f"   Color: {comparison['color_similarity']:.3f}")
            print(f"   Brightness: {comparison['brightness_similarity']:.3f}")
            print(f"   Overall: {comparison['overall_similarity']:.3f}")
            
            # Move vehicle to new location for variety
            if self.carla.vehicle:
                spawn_points = self.carla.world.get_map().get_spawn_points()
                new_location = random.choice(spawn_points)
                self.carla.vehicle.set_transform(new_location)
                time.sleep(1.0)  # Wait for vehicle to settle
        
        # Calculate average results
        if results:
            avg_results = {
                'avg_ssim': np.mean([r['ssim'] for r in results]),
                'avg_color': np.mean([r['color_similarity'] for r in results]),
                'avg_brightness': np.mean([r['brightness_similarity'] for r in results]),
                'avg_overall': np.mean([r['overall_similarity'] for r in results])
            }
            
            print(f"\n📈 COMPARISON RESULTS SUMMARY:")
            print(f"   Average SSIM: {avg_results['avg_ssim']:.3f}")
            print(f"   Average Color Similarity: {avg_results['avg_color']:.3f}")
            print(f"   Average Brightness Similarity: {avg_results['avg_brightness']:.3f}")
            print(f"   Average Overall Similarity: {avg_results['avg_overall']:.3f}")
            
            return avg_results
        
        return None

# ===== PART 4: TRAINING DATA AUGMENTATION =====
class TrainingDataAugmentation:
    """Use SDXL data to augment CARLA training datasets"""
    
    def __init__(self, sdxl_loader):
        self.sdxl = sdxl_loader
    
    def create_mixed_dataset(self, carla_images, mix_ratio=0.3):
        """Create mixed dataset with CARLA and SDXL images"""
        
        num_sdxl_images = int(len(carla_images) * mix_ratio)
        
        # Select random SDXL images
        sdxl_indices = random.sample(range(len(self.sdxl.images)), 
                                   min(num_sdxl_images, len(self.sdxl.images)))
        
        mixed_dataset = []
        labels = []
        
        # Add CARLA images
        for img in carla_images:
            mixed_dataset.append(img)
            labels.append('CARLA')
        
        # Add SDXL images
        for idx in sdxl_indices:
            mixed_dataset.append(self.sdxl.images[idx])
            labels.append('SDXL')
        
        print(f"📊 Created mixed dataset:")
        print(f"   CARLA images: {len(carla_images)}")
        print(f"   SDXL images: {len(sdxl_indices)}")
        print(f"   Total: {len(mixed_dataset)}")
        
        return mixed_dataset, labels
    
    def apply_domain_adaptation(self, sdxl_image, target_brightness=0.45):
        """Apply domain adaptation to make SDXL images more CARLA-like"""
        
        # Adjust brightness to match CARLA
        current_brightness = np.mean(sdxl_image) / 255.0
        brightness_factor = target_brightness / current_brightness
        
        adapted = sdxl_image.astype(np.float32) * brightness_factor
        adapted = np.clip(adapted, 0, 255).astype(np.uint8)
        
        # Apply slight blur to match CARLA rendering
        adapted = cv2.GaussianBlur(adapted, (3, 3), 0.5)
        
        return adapted

# ===== PART 5: MAIN USAGE EXAMPLE =====
def main_carla_integration_example():
    """Complete example of CARLA-SDXL integration"""
    
    print("🚀 CARLA-SDXL INTEGRATION EXAMPLE")
    print("=" * 40)
    
    # Step 1: Setup CARLA integration
    carla_integration = CARLAIntegration()
    
    if not carla_integration.connect_to_carla():
        print("❌ Cannot connect to CARLA. Make sure CARLA is running.")
        return
    
    # Step 2: Load SDXL dataset
    # Replace with your actual dataset path
    dataset_path = "sdxl_rural_dataset_20241201_143022"  # Example path
    
    if not carla_integration.load_sdxl_dataset(dataset_path):
        print("❌ Cannot load SDXL dataset. Check path.")
        return
    
    # Step 3: Setup vehicle and camera
    if not carla_integration.spawn_vehicle():
        return
    
    if not carla_integration.setup_camera():
        return
    
    # Step 4: Run comparison
    comparator = DataComparison(carla_integration, carla_integration.sdxl_loader)
    results = comparator.run_comparison_session(num_comparisons=5)
    
    # Step 5: Create augmented dataset
    augmenter = TrainingDataAugmentation(carla_integration.sdxl_loader)
    
    if comparator.carla_images:
        mixed_dataset, labels = augmenter.create_mixed_dataset(
            comparator.carla_images, 
            mix_ratio=0.4
        )
        
        print(f"✅ Created augmented training dataset with {len(mixed_dataset)} images")
    
    # Cleanup
    if carla_integration.camera:
        carla_integration.camera.destroy()
    if carla_integration.vehicle:
        carla_integration.vehicle.destroy()
    
    print("✅ CARLA integration example complete!")

# ===== USAGE INSTRUCTIONS =====
print("\n💡 USAGE INSTRUCTIONS:")
print("=" * 25)
print("1. Install CARLA simulator and Python API:")
print("   pip install carla")
print("   # Download CARLA from: https://carla.org/")
print()
print("2. Start CARLA simulator:")
print("   ./CarlaUE4.sh  # Linux")
print("   # or CarlaUE4.exe on Windows")
print()
print("3. Run integration:")
print("   # Replace dataset path with your actual path")
print("   main_carla_integration_example()")
print()
print("4. Use cases:")
print("   • Compare SDXL vs CARLA image quality")
print("   • Augment CARLA training data with SDXL images")
print("   • Validate SDXL realism against simulation")
print("   • Create mixed synthetic datasets")

print("\n🎯 INTEGRATION BENEFITS:")
print("=" * 25)
print("✅ Validate SDXL quality against CARLA simulation")
print("✅ Augment limited CARLA data with diverse SDXL scenes")
print("✅ Compare synthetic data generation approaches")
print("✅ Create larger, more diverse training datasets")
print("✅ Test domain adaptation techniques")

if __name__ == "__main__":
    # Uncomment to run the example
    # main_carla_integration_example()
    pass